# Regresión lineal simple

Es un método que busca modelar la relación entre una varieble independiente **`x`** y una variable dependiente **`y`**, de un conjunto de **`m`** datos conocidos, a travéz de una función líneal:

$$ f(x_{i}) = w * x_{i} + b = \hat{y_{i}} $$
$$ i = 1, ..., m $$

De esta forma podremos realizar una predicción de los valores de **`y`** para nuevos valores de **`x`**.

![Datos y función lineal modelada](images/introduction.png)

El objetivo es encontrar los parametros **`w`** (peso o weight en inglés) y **`b`** (parcialidad o bias en inglés), que formen la función lineal que **mejor se adapte a los datos**, a partir de los mismos. Pero primero necesitamos una forma de cuantificar que tan bien se adapta, o explica los datos, una función lineal cualquiera.

## Función de costo

Para darle un valor al ajuste de una recta con respecto a ciertos datos, vamos a contruir una linea cualquiera y graficarla junto al conjunto de datos que queremos modelar.

![Datos y función lineal de prueba](images/data_and_one_linear_function.png)

Ahora vamos a definir los errores $ \varepsilon_{i} $ como la diferencia entre el valor real $ y_{i} $ y el valor de la linea $ \hat{y_{i}} $ para cada $ x_{i} $ del conjunto de datos.

$$ \varepsilon_{i} = y_{i} - \hat{y_{i}} = y_{i} - (w * x_{i} + b) $$
$$ i = 1, ..., m $$

![Datos y errores](images/dsfadsgf.png)

Estos errores nos pueden dar una intuición de qué tan bien se ajusta una recta a un conjunto de datos, por ejemplo si sumamos a todos. Rectas que se adapten mejor tendrán una suma de errores menor, y rectas que se adapten peor tendán una suma de errores mayor. Pero como se puede observar en la figura, los errores pueden ser positivos o negativos, lo cual hace que la suma no refleje realmente el error, o la distancia entre la recta y los datos.
Para mejorar nuestra forma de valorizar el ajuste de una recta, vamos a definir la **función de pérdida** (loss function en inglés) como la mitad de el error al cuadrado.

$$ \ell_{i} = \frac{1}{2} * ( y_{i} - \hat{y_{i}} )^2 = \frac{1}{2} * ( \varepsilon_{i} )^2 $$
$$ i = 1, ..., m $$

Ventajas:
 - Al elevar el error al cuadrado, me aseguro que son todos positivos, y contribuirán a la suma de errores siempre aumentandola.
 - Al elevar el error al cuadrado tabién estoy haciendo que los errores más grandes incidán todavía más sobre la suma de errores, mientras que errores más chicos contribuirán menos al error total.
 - El echo de usar la mitad del error al cuadrado servirá exclusivamente a la hora de derivar la función para hacer dicho cálculo más sencillo.
 
Por último vamos a definir la función de costo de una recta con respecto a una serie de datos, como el promedio de las **pérdidas** de cada punto de los datos. Esta función representa el **ajuste** o la **distancia** de la recta con respecto a los datos. Mientras menor sea el costo de una recta, mejor va a ser esta para explicar el comportamiento de la variable dependiente **`y`** con respecto a la variable independiente **`x`** de dicho conjunto de datos.

$$ C = \frac{1}{m} * \displaystyle\sum_{i=1}^{m} \frac{1}{2} * ( y_{i} - \hat{y_{i}} )^2 $$

https://mickteaching.wordpress.com/2016/04/19/data-need-to-be-normally-distributed-and-other-myths-of-linear-regression/
https://stats.stackexchange.com/questions/148803/how-does-linear-regression-use-the-normal-distribution
https://en.wikipedia.org/wiki/Simple_linear_regression
https://www.researchgate.net/post/Is_linear_regression_valid_when_the_outcome_dependant_variable_not_normally_distributed

### Linea de mejor *ajuste*

Explicar porque el echo de que los errores con respecto a la linea esten normalmente distribuídos hace que sea un buen modelo para predecir el mundo.


El objetivo es encontrar un valor para los parametros **`w`** y **`b`** que hagan que la función lineal se *ajuste* lo mejor posible a los datos. Esto significa que los errores de cada dato conocido, con respecto a la predicción efectuada por la función, se encuentren en una distribución normal con media 0 ($ \mu = 0 $).

![Datos y errores](images/data_and_errors.png)